## Import Libraries, Define Paths

In [1]:
import pandas as pd
import numpy as np
import os

base = '/Users/jetcalz07/Desktop/MIDS/W241_Experiments/project/'
data = base + 'data/'

## Load Data

In [2]:
# Filter columns from extract, rename
keep_cols = ['IPAddress', 'Random ID', 'Q26', 'Q2', 'Q27', 'Q28', 'Q3', 'Q4', 'Q7', 'Q8', 'Q9', 'Q10', 
'Message_Received', 'Shelter_Information', 'Shelter_Type_1', 'Shelter_Type_2', 'Shelter_Type_3', 'Shelter_Type_4', 
'Q74_1', 'Q74_2', 'Q74_3', 'Q74_4', 'Q75_1', 'Q75_2', 'Q75_3', 'Q75_4', 'Q76_1', 'Q76_2', 'Q76_3', 'Q76_4',
'Small_1', 'Small_2', 'Small_3', 'Small_4', 'Medium_1', 'Medium_2', 'Medium_3', 'Medium_4', 
'Large_1', 'Large_2', 'Large_3', 'Large_4', 'Duration (in seconds)']
rename_cols = ['ip', 'rid', 'rid_pasted', 'age_bin', 'gender', 'marital', 'income_bin', 'state',
'own_dog', 'dog_or_cat', 'adopt_or_shop', 'dog_size',  'msg_treat_ind', 'shelter_treat_bin',
'shelter_tp_1', 'shelter_tp_2', 'shelter_tp_3', 'shelter_tp_4', 'sm_1_ns', 'sm_2_ns', 'sm_3_ns', 'sm_4_ns',
'med_1_ns', 'med_2_ns', 'med_3_ns', 'med_4_ns', 'lg_1_ns', 'lg_2_ns', 'lg_3_ns', 'lg_4_ns', 'sm_1_ys', 'sm_2_ys', 'sm_3_ys', 'sm_4_ys',
'med_1_ys', 'med_2_ys', 'med_3_ys', 'med_4_ys', 'lg_1_ys', 'lg_2_ys', 'lg_3_ys', 'lg_4_ys', 'duration']
trim_cols =  ['ip', 'rid', 'age_bin', 'gender', 'marital', 'income_bin', 'state',
'own_dog', 'dog_or_cat', 'adopt_or_shop', 'dog_size',  'msg_treat_ind', 'shelter_treat_bin', 'duration']
desired_cols =  trim_cols + ['dog_num', 'rating', 'shelter_tp']

In [3]:
# Load relevant columns from Qualtrics output, trim fat, rename
fname = 'das_final_results.csv'
df_init = pd.read_csv(data+fname, usecols=keep_cols)
df_init.rename(columns=dict(zip(keep_cols, rename_cols)), inplace=True)
df_init.reset_index(inplace=True, drop=True)
df_init.reset_index(level = 0,inplace=True)
print(f"No. participants: {len(df_init)}")
df_init.head()

No. participants: 880


,index,ip,duration,age_bin,gender,marital,income_bin,state,own_dog,dog_or_cat,...,lg_3_ys,lg_4_ys,rid_pasted,rid,msg_treat_ind,shelter_treat_bin,shelter_tp_1,shelter_tp_2,shelter_tp_3,shelter_tp_4
0,0,104.175.0.154,88,45-54 years old,Male,Single,"$150,000 or more",California,"No, but I own another pet",Neither dogs nor cats,...,NaN,NaN,924675,924675,No,Yes,Traditional Shelter,Traditional Shelter,No-Kill Shelter,No-Kill Shelter
1,1,104.37.172.149,245,25-34 years old,Female,Single,"$50,000-$74,999",North Carolina,Yes,Dogs,...,NaN,NaN,NaN,276938,Yes,Yes,No-Kill Shelter,No-Kill Shelter,No-Kill Shelter,No-Kill Shelter
2,2,128.254.166.169,94,25-34 years old,Male,Married,"$50,000-$74,999",Indiana,Yes,Dogs,...,NaN,NaN,185377,185377,No,No,No-Kill Shelter,Traditional Shelter,No-Kill Shelter,Traditional Shelter
3,3,137.148.112.26,79,35-44 years old,Female,Single,"$50,000-$74,999",Ohio,Yes,Dogs,...,NaN,NaN,864738,864738,Yes,Yes,Traditional Shelter,No-Kill Shelter,No-Kill Shelter,No-Kill Shelter
4,4,206.82.9.171,86,25-34 years old,Male,Single,"$75,000-$99,999",Pennsylvania,Yes,Dogs,...,NaN,NaN,104858,104858,Yes,No,No-Kill Shelter,Traditional Shelter,Traditional Shelter,Traditional Shelter


In [4]:
## Create features at participant level
# 1. Check if rids match within Qualtrics (not essential but may help)
df_init['rid_match_qualtrics'] = df_init['rid'].astype(str)==df_init['rid_pasted'].astype(str)
#print(round(np.mean(df_init['rid_match_qualtrics']), 3))

# Check for nulls
df_init.dropna(subset=['gender', 'marital'], how='any', inplace=True)
nulls = pd.DataFrame(df_init.isnull().sum()).sort_values(0, ascending=False)
#nulls.loc[nulls[0]>0]

## Wrangle Data

In [5]:
# Init DF
df = pd.DataFrame(columns=desired_cols)

# Loop through rows
idx = 0
for idx, row in df_init.iterrows():
    base_row = row[trim_cols].tolist()

    for i in range(1, 5):
        sizes = row[[f'sm_{i}_ns', f'med_{i}_ns', f'lg_{i}_ns', f'sm_{i}_ys', f'med_{i}_ys', f'lg_{i}_ys']]
        rating = str(sizes[~sizes.isnull()][0])
        if row['shelter_treat_bin'] == 'No':
            shelter_tp = None
        else:
            shelter_tp = row[f'shelter_tp_{i}']
        new_row = base_row.copy() + [i, rating, shelter_tp]

        # save out
        df.loc[len(df), ] = new_row

# display
print(len(df))
df.head()

3428


,ip,rid,age_bin,gender,marital,income_bin,state,own_dog,dog_or_cat,adopt_or_shop,dog_size,msg_treat_ind,shelter_treat_bin,duration,dog_num,rating,shelter_tp
0,104.175.0.154,924675,45-54 years old,Male,Single,"$150,000 or more",California,"No, but I own another pet",Neither dogs nor cats,I will adopt a dog,Medium (Between 30-60 lbs.),No,Yes,88,1,8,Traditional Shelter
1,104.175.0.154,924675,45-54 years old,Male,Single,"$150,000 or more",California,"No, but I own another pet",Neither dogs nor cats,I will adopt a dog,Medium (Between 30-60 lbs.),No,Yes,88,2,9,Traditional Shelter
2,104.175.0.154,924675,45-54 years old,Male,Single,"$150,000 or more",California,"No, but I own another pet",Neither dogs nor cats,I will adopt a dog,Medium (Between 30-60 lbs.),No,Yes,88,3,8,No-Kill Shelter
3,104.175.0.154,924675,45-54 years old,Male,Single,"$150,000 or more",California,"No, but I own another pet",Neither dogs nor cats,I will adopt a dog,Medium (Between 30-60 lbs.),No,Yes,88,4,10 (high),No-Kill Shelter
4,104.37.172.149,276938,25-34 years old,Female,Single,"$50,000-$74,999",North Carolina,Yes,Dogs,I will shop for a dog,Small (Less than 30 lbs.),Yes,Yes,245,1,3,No-Kill Shelter


## Convert Data Types (Str to Int)

In [6]:
# Rename values
df.loc[df['rating']=='10 (high)', 'rating'] = '10'
df.loc[df['rating']=='1 (low)', 'rating'] = '1'
df['rating'] = df['rating'].astype(int) # Convert ratings to int

# Rename col strings to integers with dict
dict_df = pd.DataFrame(columns=['field', 'string', 'value'])

age_dict = {'Under 18 years': '0',
 '18-24 years old': '1',
 '25-34 years old': '2',
 '35-44 years old': '3',
 '45-54 years old': '4',
 '55-64 years old': '5',
 '65+ years old': '6'}
income_dict = {'Less than $25,000': 0,
 '$25,000-$49,999': 1,
 '$50,000-$74,999': 2,
 '$75,000-$99,999': 3,
 '$100,000-$149,000': 4,
 '$150,000 or more': 5}
size_dict = {'Small (Less than 30 lbs.)': 0, 'Medium (Between 30-60 lbs.)': 1, 'Large (Greater than 60 lbs.)': 2}
shelter_dict = {None: 0, 'Traditional Shelter': 1, 'No-Kill Shelter': 2}
df.replace({
    'age_bin': age_dict,
    'income_bin': income_dict,
    'dog_size': size_dict,
    'shelter_tp': shelter_dict
    }, inplace=True)

# Save mapping to dict for reference
cols = ['age_bin', 'income_bin', 'dog_size', 'shelter_tp']
col_dicts = [age_dict, income_dict, size_dict, shelter_dict]
for i in range(len(col_dicts)):
    # Save to dict df mapping
    dict_df2 = pd.DataFrame({'field': [cols[i] for x in range(len(col_dicts[i].keys()))], 'string': col_dicts[i].keys(), 'value':col_dicts[i].values()})
    dict_df = pd.concat([dict_df, dict_df2], axis=0)

# All others (order not as important)
cvt_cols = ['gender', 'marital', 'state', 'own_dog', 'dog_or_cat',
'adopt_or_shop', 'msg_treat_ind', 'shelter_treat_bin']

for col in cvt_cols:
    values = list(np.sort(df[col].unique()))
    col_dict = dict(zip(values, range(0, len(values))))
    df.replace({col: col_dict}, inplace=True)

    # Save to dict df mapping
    dict_df2 = pd.DataFrame({'field': [col for x in range(len(col_dict.keys()))], 'string': col_dict.keys(), 'value':col_dict.values()})
    dict_df = pd.concat([dict_df, dict_df2], axis=0)

df.head(3)

,ip,rid,age_bin,gender,marital,income_bin,state,own_dog,dog_or_cat,adopt_or_shop,dog_size,msg_treat_ind,shelter_treat_bin,duration,dog_num,rating,shelter_tp
0,104.175.0.154,924675,4,1,2,5,4,1,2,0,1,0,1,88,1,8,1
1,104.175.0.154,924675,4,1,2,5,4,1,2,0,1,0,1,88,2,9,1
2,104.175.0.154,924675,4,1,2,5,4,1,2,0,1,0,1,88,3,8,2


In [10]:
display(dict_df)

,field,string,value
0,age_bin,Under 18 years,0
1,age_bin,18-24 years old,1
2,age_bin,25-34 years old,2
3,age_bin,35-44 years old,3
4,age_bin,45-54 years old,4
...,...,...,...
1,adopt_or_shop,I will shop for a dog,1
0,msg_treat_ind,No,0
1,msg_treat_ind,Yes,1
0,shelter_treat_bin,No,0


## Save Out

In [11]:
df.to_csv(data+'das_results_clean.csv', index=False)
dict_df.to_csv(data+'das_value_map.csv', index=False)

### Convert Categorical to Dummy (If Needed)

In [ ]:
# Convert categorical to dummy
#pd.get_dummies(df['own_dog'], prefix='own_dog', drop_first=True)
#pd.get_dummies(data=df, columns=['own_dog'], prefix=['own_dog'], drop_first=True)

### Analysis Notes
- Run LR on the full dataset (Create interaction terms with treatments)
- Key areas to investigate
    - What had the greatest effect on ratings?
    - Did the message treatment alone increase ratings?
    - Did the shelter tp treatment alone increase?
    - Did the interaction of treatments increase? Which was strongest?
    - Which covariates were most impactful?
        - Ex: did the survey completion time effect ratings?
- Run LR on subsets of features to get a set of models 
- Run F-Test acros models to see which models are most predictive/significant